In [45]:
import malariagen_data
import allel
import dask

In [2]:
%run "../malariagen_data/util.py"

In [3]:
ag3 = malariagen_data.Ag3()

In [4]:
contig_dict = {
    "2R": 0, 
    "2L": 1,
    "3R": 2,
    "3L": 3,
    "X" : 4
}

In [7]:
aims = "gamb_vs_colu"
sample_sets = "AG1000G-UG"

In [9]:
def aim_calls(aims, sample_sets, sample_query=None):
    
    # contig dict
    contig_dict = {
    "2R": 0, 
#     "2L": 1,
#     "3R": 2,
#     "3L": 3,
#     "X" : 4
    }
    
    # output dict
    ds_dict = {}
    
    # or aim_calls
    aim_sites = ag3.aim_sites(aims=aims)
    aim_sites = aim_sites.set_index(variants='variant_contig')
    
    # get calls for each aim
    for c in contig_dict:
        print(f"collecting {c} aim calls")
        # aim alleles and positions
        ca_sites = aim_sites.sel(variants=contig_dict[c])
        ref = ca_sites['variant_allele'][:,0].values
        alt = ca_sites['variant_allele'][:,1].values
        ca_pos = ca_sites['variant_position'].values
        # snps at aim positions
        snp_calls = ag3.snp_calls(region=c, sample_sets=sample_sets)
        ca_calls = snp_calls.set_index(variants='variant_position')
        sel_calls = ca_calls.sel(variants=ca_pos)
#         alleles = sel_calls['variant_allele'][:].values
#         # map aim/snp alleles
#         mapping = allel.create_allele_mapping(ref, alt, alleles)
#         gt = allel.GenotypeDaskArray(sel_calls['call_genotype'].data)
#         gt_map = gt.map_alleles(mapping)
        
#         ds_dict[c] = ca_calls.sel(variants=ca_pos)
        
#     # concat at this point?    
#     ds_concat = xarray_concat([ds_dict[c] for c in contig_dict], "variants")



    aim_calls['call_genotype'] = (("variants", "samples", "ploidy"), concat_dask_gt)
        
    return aim_calls

In [10]:
%%time
ca_sites, sel_calls = aim_calls(aims=aims, sample_sets=sample_sets)

collecting 2R aim calls
CPU times: user 6.04 s, sys: 1.93 s, total: 7.97 s
Wall time: 11.8 s


In [31]:
aim_alleles = ca_sites['variant_allele'].values

In [32]:
snp_alleles = sel_calls['variant_allele'][:].values

In [33]:
snp_ref = snp_alleles[:,0]

In [34]:
snp_alt = snp_alleles[:,1:]

In [35]:
mapping = allel.create_allele_mapping(snp_ref, snp_alt, aim_alleles)

In [36]:
mapping

array([[ 0, -1, -1,  1],
       [ 1,  0, -1, -1],
       [ 1, -1, -1,  0],
       [ 1, -1,  0, -1],
       [ 1, -1, -1,  0],
       [ 1, -1, -1,  0],
       [ 1, -1, -1,  0],
       [ 1,  0, -1, -1],
       [ 1, -1, -1,  0],
       [ 1, -1, -1,  0],
       [ 1,  0, -1, -1],
       [ 1, -1, -1,  0],
       [ 1, -1,  0, -1],
       [ 0, -1,  1, -1],
       [ 0, -1,  1, -1],
       [ 1, -1, -1,  0],
       [ 1, -1, -1,  0],
       [ 1, -1,  0, -1],
       [ 1, -1, -1,  0],
       [ 1, -1,  0, -1],
       [ 1, -1, -1,  0],
       [ 1, -1,  0, -1],
       [ 1, -1, -1,  0],
       [ 0, -1, -1,  1],
       [ 1,  0, -1, -1],
       [ 1, -1, -1,  0],
       [ 1, -1,  0, -1],
       [ 1, -1,  0, -1],
       [ 0, -1,  1, -1],
       [ 0, -1,  1, -1],
       [ 0, -1,  1, -1],
       [ 0, -1,  1, -1],
       [ 1, -1,  0, -1],
       [ 0, -1, -1,  1],
       [ 1, -1, -1,  0],
       [ 1,  0, -1, -1],
       [ 1, -1,  0, -1],
       [ 1,  0, -1, -1],
       [ 1, -1,  0, -1],
       [ 1, -1, -1,  0],


In [37]:
gt = allel.GenotypeDaskArray(sel_calls['call_genotype'].data)

In [38]:
gt

<GenotypeDaskArray shape=(58, 290, 2) dtype=int8>

In [39]:
gt_map = gt.map_alleles(mapping)
gt_map

<GenotypeDaskArray shape=(58, 290, 2) dtype=int8>

In [49]:
type(gt_map.values)

dask.array.core.Array

In [ ]:
dask_concat([gt_map.data, ...,..])

In [47]:
dask_concat()

NameError: name 'dask_concat' is not defined

In [11]:
ref = ca_sites['variant_allele'][:,0].values

In [12]:
alt = ca_sites['variant_allele'][:,1].values

In [13]:
alleles = sel_calls['variant_allele'][:].values

In [14]:
ref

array([b'A', b'C', b'G', b'T', b'G', b'G', b'G', b'C', b'T', b'G', b'C',
       b'G', b'T', b'C', b'A', b'G', b'G', b'T', b'G', b'T', b'G', b'T',
       b'T', b'A', b'C', b'G', b'T', b'T', b'C', b'C', b'C', b'A', b'T',
       b'A', b'G', b'C', b'T', b'C', b'T', b'T', b'G', b'A', b'T', b'G',
       b'A', b'G', b'C', b'T', b'C', b'T', b'G', b'C', b'A', b'C', b'T',
       b'T', b'T', b'T'], dtype='|S1')

In [15]:
alt

array([b'G', b'A', b'A', b'A', b'A', b'A', b'A', b'A', b'G', b'A', b'A',
       b'A', b'C', b'T', b'T', b'A', b'A', b'A', b'C', b'C', b'A', b'C',
       b'G', b'G', b'A', b'C', b'A', b'A', b'T', b'T', b'T', b'T', b'A',
       b'G', b'A', b'A', b'C', b'A', b'C', b'G', b'A', b'G', b'A', b'T',
       b'T', b'A', b'G', b'G', b'T', b'G', b'C', b'A', b'G', b'T', b'A',
       b'C', b'C', b'A'], dtype='|S1')

In [16]:
alleles

array([[b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'

In [17]:
len(alt)

58

In [15]:
mapping = allel.create_allele_mapping(alleles, )

In [16]:
mapping

array([[0, 3],
       [1, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [3, 0],
       [3, 0],
       [1, 0],
       [3, 0],
       [3, 0],
       [1, 0],
       [3, 0],
       [2, 0],
       [0, 2],
       [0, 2],
       [3, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [2, 0],
       [3, 0],
       [0, 3],
       [1, 0],
       [3, 0],
       [2, 0],
       [2, 0],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [2, 0],
       [0, 3],
       [3, 0],
       [1, 0],
       [2, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [3, 0],
       [0, 3],
       [2, 0],
       [0, 3],
       [0, 2],
       [3, 0],
       [0, 3],
       [3, 0],
       [0, 2],
       [3, 0],
       [3, 0],
       [1, 0],
       [0, 3],
       [0, 2],
       [2, 0],
       [2, 0],
       [2, 0],
       [2, 0]], dtype=int8)

In [17]:
gt = allel.GenotypeDaskArray(sel_calls['call_genotype'].data)

In [18]:
gt[:10,:10]

<GenotypeDaskArray shape=(10, 10, 2) dtype=int8>

In [19]:
gt_map = gt.map_alleles(mapping)

In [20]:
gt_map[:10,:10]

<GenotypeDaskArray shape=(10, 10, 2) dtype=int8>

In [ ]:
snp_calls = ag3.snp_calls(region='2R', sample_sets="3.0")

In [ ]:
snp_calls['variant_allele'][:1,:5].values

In [ ]:
aims = "gamb_vs_colu"
sample_sets = "AG1000G-UG"

In [ ]:
aim_sites = ag3.aim_sites(aims=aims)

In [ ]:
aim_sites

In [ ]:
aim_sites = aim_sites.set_index(variants='variant_contig')

In [ ]:
ag3

In [ ]:
zero = aim_sites.sel(variants=0)

In [ ]:
zero_pos = zero['variant_position'].values

In [ ]:
region = '2R'

In [ ]:
snp_calls = ag3.snp_calls(region=region, sample_sets="3.0")

In [ ]:
snp_calls

In [ ]:
snp_calls = snp_calls.set_index(variants='variant_position')

In [ ]:
ds_dict= {}

In [ ]:
%%time
ds_dict[0] = snp_calls.sel(variants=zero_pos)
ds_dict[0]

In [ ]:
%%time
one = aim_sites.sel(variants=1)
one_pos = one['variant_position'].values
region = '2L'
snp_calls = ag3.snp_calls(region=region, sample_sets="3.0")
snp_calls = snp_calls.set_index(variants='variant_position')

In [ ]:
snp_calls = ag3.snp_calls(region=region, sample_sets="3.0")

In [ ]:
%%time
snp_calls = snp_calls.set_index(variants='variant_position')

In [ ]:
%%time
ds_dict[1] = snp_calls.sel(variants=one_pos)
ds_dict[1]

In [ ]:
test_concat = xarray_concat([ds_dict[0], ds_dict[1]], "variants")

In [ ]:
test_concat

In [ ]:
test_concat["call_genotype"][:1].values

In [ ]:
for c in contig_dict.keys():
    print(c)
    

In [ ]:
contig_dict = {
    "2R": 0, 
    "2L": 1,
    "3R": 2,
    "3L": 3,
    "X" : 4
}

In [ ]:
snp_calls.sel?